In [1]:
import dspy
from dspy.datasets import DataLoader
from dspy.evaluate import Evaluate

from dspy.teleprompt import MIPROv2


import os
from dotenv import load_dotenv

from typing import Optional
import random
import numpy as np
import pandas as pd
import warnings
from collections import defaultdict
from tqdm import tqdm

from huggingface_hub import login

load_dotenv()
warnings.filterwarnings('ignore')

In [3]:
train=pd.read_json('../data/train.json')
dl = DataLoader()

In [8]:
train


,uri,question,context,answers,labelled_answer_spans,labelled_summaries,raw_text
0,4367393,what is parkinesonism?,,[u spelt it wrong !!\nParkinson's disease is o...,{'INFORMATION': [{'txt': 'Parkinson's disease ...,{'INFORMATION_SUMMARY': 'Parkinson's disease i...,uri: 4367393\nquestion: what is parkinesonism?...
1,1504599,"I scream, shout and swear in my sleep. How do ...",My mom always told me that I talk a lot in my ...,[hmm i would say duck tape but thats a lil to ...,"{'SUGGESTION': [{'txt': 'duck tape', 'label_sp...",{'CAUSE_SUMMARY': 'Shouting during sleep may b...,"uri: 1504599\nquestion: I scream, shout and sw..."
2,1701378,what does m.c.h.c stand for in my hematology t...,,"[Mean Corpuscular Hemoglobin Concentration\n""T...",{'INFORMATION': [{'txt': ': Mean Corpuscular H...,{'INFORMATION_SUMMARY': 'For information purpo...,uri: 1701378\nquestion: what does m.c.h.c stan...
3,45070,whats the best way to clear pluged up sinuses?,,[i use a nasal bulb and saline water. 1/8 teas...,{'SUGGESTION': [{'txt': 'i use a nasal bulb an...,{'SUGGESTION_SUMMARY': 'One individual recomme...,uri: 45070\nquestion: whats the best way to cl...
4,1752893,Are their any good home remedies for tooth pain?,,[Clove Oil from a health store works really we...,{'SUGGESTION': [{'txt': 'try putting a wet tea...,"{'EXPERIENCE_SUMMARY': 'In users expereince, o...",uri: 1752893\nquestion: Are their any good hom...
...,...,...,...,...,...,...,...
2231,1371224,How can I relieve the pain of a strained bicep...,Looking for help relieving pain from a striane...,[Heat will help the muscle relax. Use that for...,{'SUGGESTION': [{'txt': 'Heat will help the mu...,{'SUGGESTION_SUMMARY': 'Apply heat to relax th...,uri: 1371224\nquestion: How can I relieve the ...
2232,512601,Would you end a possible serious relationship ...,,[Depends on if I was in loe with them. If the...,{'SUGGESTION': [{'txt': 'Depends on if I was i...,"{'EXPERIENCE_SUMMARY': 'In users experience, a...",uri: 512601\nquestion: Would you end a possibl...
2233,1038896,Hearing aids furnished to service connected ve...,brands and types,[Most of the hearing aids I see that were obta...,{'INFORMATION': [{'txt': 'Most of the hearing ...,{'INFORMATION_SUMMARY': 'Many veterans receive...,uri: 1038896\nquestion: Hearing aids furnished...
2234,1113341,How do I quickly get rid of pimples?,"I have tried all the basic creams, scrubs and ...","[a dab or glob , whichever you prefer, of toot...","{'SUGGESTION': [{'txt': 'a dab or glob , which...",{'SUGGESTION_SUMMARY': 'Dab or glob a small am...,uri: 1113341\nquestion: How do I quickly get r...


In [13]:
def llm_dataset(data: pd.DataFrame, 
                path: Optional[str]=None):
        
    

    if os.path.exists(path):
        
        llm_dataset = dl.from_csv(
            path,
            input_keys=('question','answers','perspective','perspective_definition','tone_attribute')
        )
        
        return llm_dataset
                    
    else:
        
        df_llm=defaultdict(list)

        
        for idx,row in data.iterrows():
        
            summary_lbls=len(row['labelled_summaries'].items())

            ans=' '.join(a for a in row['answers'])
            ans=[ans]*summary_lbls
            ques=[row['question']]*summary_lbls
            
            df_llm['question'].extend(ques)
            df_llm['answers'].extend(ans)
                    
            for (keys,values) in row['labelled_summaries'].items():
                
                df_llm['summaries'].append(values)
                
                if 'information' in keys.lower():
                    df_llm['perspective'].append('information')
                    tone_attribute = "Informative, Educational"

                    defn = "Defined as knowledge about diseases, disorders, and health-related facts, providing insights into symptoms and diagnosis."
                    
                    df_llm['perspective_definition'].append(defn)
                    df_llm['tone_attribute'].append(tone_attribute)

                    
                elif 'cause' in keys.lower():
                    df_llm['perspective'].append('cause')
                    tone_attribute = "Advisory, Recommending"
                    defn = "Defined as reasons responsible for the occurrence of a particular medical condition, symptom, or disease"
                    
                    df_llm['perspective_definition'].append(defn)
                    df_llm['tone_attribute'].append(tone_attribute)



                elif 'suggestion' in keys.lower():
                    df_llm['perspective'].append('suggestion')
                    tone_attribute = "Advisory, Recommending"
                    defn = "Defined as advice or recommendations to assist users in making informed medical decisions, solving problems, or improving health issues."
                    
                    df_llm['perspective_definition'].append(defn)
                    df_llm['tone_attribute'].append(tone_attribute)


                    
                elif 'experience' in keys.lower():
                    df_llm['perspective'].append('experience')
                    tone_attribute = "Personal, Narrative"

                    defn = "Defined as individual experiences, anecdotes, or firsthand insights related to health, medical treatments, medication usage, and coping strategies"

                    df_llm['perspective_definition'].append(defn)
                    df_llm['tone_attribute'].append(tone_attribute)

                else:
                    df_llm['perspective'].append('question')
                    tone_attribute = "Seeking Understanding"

                    defn = "Defined as inquiry made for deeper understanding."
                    
                    df_llm['perspective_definition'].append(defn)
                    df_llm['tone_attribute'].append(tone_attribute)
        
        
        
        pd.DataFrame(dict(df_llm)).to_csv(path,index=False)
        
        llm_dataset = dl.from_csv(
            path,
            input_keys=('question','answers','perspective','perspective_definition','tone_attribute')            
        )
        
        return llm_dataset

trainset=llm_dataset(train,'../data/llm_train.csv')

In [ ]:
"""hf_token = os.environ.get("HF_API_KEY")
login(hf_token)"""

llm = dspy.LM(model='ollama/llama3.2',max_tokens=10000)
dspy.configure(lm=llm)

In [21]:
class Summary(dspy.Signature):
    """
    Summarize the following content according to the perspective and the asked question. The abstractive summary created should detect the spans in the different answers describing the particular perspective. Utilize the definition of the perspective and the tone information for accurate summary generation.
    """
    question: str = dspy.InputField(desc="The question")
    answers: str = dspy.InputField(desc="The answers to the question asked which needs to be summarized")
    
    perspective: str = dspy.InputField(desc="The Perspective for which the summary needs to attuned to")
    
    perspective_definition: str = dspy.InputField(desc="defines the semantic of the particular perspective that helps the model understand the specific medical context")
    
    tone_attribute: str = dspy.InputField(desc="The tone reflects the stylistic approach the summary should take.")
    
    gen_summary: str= dspy.OutputField(desc="The generated summary")

## Metrics

In [22]:
class DecompositionalSemanticRecallPrecision(dspy.Signature):
    """
    Compare a system's response to the ground truth to compute recall and precision of key ideas.
    You will first enumerate key ideas in each response, discuss their overlap, and then report recall and precision.
    """

    question: str = dspy.InputField()
    ground_truth: str = dspy.InputField()
    system_response: str = dspy.InputField()
    ground_truth_key_ideas: str = dspy.OutputField(desc="enumeration of key ideas in the ground truth")
    system_response_key_ideas: str = dspy.OutputField(desc="enumeration of key ideas in the system response")
    discussion: str = dspy.OutputField(desc="discussion of the overlap between ground truth and system response")
    recall: float = dspy.OutputField(desc="fraction (out of 1.0) of ground truth covered by the system response")
    precision: float = dspy.OutputField(desc="fraction (out of 1.0) of system response covered by the ground truth")

class SemanticRecallPrecision(dspy.Signature):
    """
    Compare a system's response to the ground truth to compute its recall and precision.
    If asked to reason, enumerate key ideas in each response, and whether they are present in the other response.
    """

    question: str = dspy.InputField()
    ground_truth: str = dspy.InputField()
    system_response: str = dspy.InputField()
    
    recall: float = dspy.OutputField(desc="fraction (out of 1.0) of ground truth covered by the system response")
    precision: float = dspy.OutputField(desc="fraction (out of 1.0) of system response covered by the ground truth")

def f1_score(precision, recall):
    precision, recall = max(0.0, min(1.0, precision)), max(0.0, min(1.0, recall))
    return 0.0 if precision + recall == 0 else 2 * (precision * recall) / (precision + recall)


class SemanticF1_Summary(dspy.Module):
    def __init__(self, threshold=0.65, decompositional=False):
        self.threshold = threshold

        if decompositional:
            self.module = dspy.ChainOfThought(DecompositionalSemanticRecallPrecision)
        else:
            self.module = dspy.ChainOfThought(SemanticRecallPrecision)

    def forward(self, example, pred, trace=None):
        
        try:
            scores = self.module(question=example.question, ground_truth=example.summaries, system_response=pred.gen_summary)
            score = f1_score(scores.precision, scores.recall)

        except:
            score=0
                
        return score if trace is None else score >= self.threshold


In [23]:
class Model(dspy.Module):
    def __init__(self):
        super().__init__()
        self.llm_summary=dspy.ChainOfThought(Summary)
        
    def forward(self,question,answers,perspective,
                perspective_definition,tone_attribute):
        summary=self.llm_summary(question=question,
                                answers=answers,
                                perspective=perspective,
                                perspective_definition=perspective_definition,
                                tone_attribute=tone_attribute,
                                )
        
        return summary

In [84]:
def inference(n:int):
    scores = []
    model=Model()
    metric_summary=SemanticF1_Summary(decompositional=False)

    for x in tqdm(trainset[:n]):
        try:
            pred = model(**x.inputs())
            scores.append(metric_summary(x,pred))
        except:
            scores.append('None')
            
    return scores
            
def sample_data(path:str,num_samps:int):
    random.seed(42)
    sample_idx=[]
    df=pd.read_csv(path)
    labels=dict(df.groupby(by='perspective').apply(len).astype(int))

    for l in labels:
        idx_list=df[df['perspective']==l].index.to_list()
        
        sample_idx.extend(random.sample(idx_list,num_samps))

        
    return df.loc[sample_idx].reset_index(drop=True)


In [119]:
tune_set=llm_dataset(sample_data('../data/llm_train.csv',20))


In [120]:
model=Model()

In [122]:
optimizer=MIPROv2(metric=SemanticF1_Summary(decompositional=False),
                prompt_model=llm,task_model=llm)

compiled_program = optimizer.compile(model, trainset=tune_set,num_trials=30, max_bootstrapped_demos=2,max_labeled_demos=2)

2025/01/13 10:24:09 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/01/13 10:24:09 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/01/13 10:24:09 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=10 sets of demonstrations...


Bootstrapping set 1/10
Bootstrapping set 2/10
Bootstrapping set 3/10


 15%|█▌        | 3/20 [00:23<02:13,  7.83s/it]


Bootstrapped 2 full traces after 3 examples for up to 1 rounds, amounting to 3 attempts.
Bootstrapping set 4/10


 10%|█         | 2/20 [00:08<01:20,  4.49s/it]


...

2025/01/13 18:14:47 INFO dspy.evaluate.evaluate: Average Metric: 9.011225565495016 / 25 (36.0%)
2025/01/13 18:14:47 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 36.04 on minibatch of size 25 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 2'].
2025/01/13 18:14:47 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [53.12, 38.81, 56.4, 38.55, 0.0, 0.0, 40.33, 40.96, 52.79, 51.23, 57.84, 52.55, 47.41, 37.6, 0.0, 54.79, 46.45, 52.47, 41.81, 38.6, 50.38, 55.52, 49.66, 37.77, 50.51, 61.92, 47.61, 57.02, 36.04]
2025/01/13 18:14:47 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [61.23, 53.33, 53.31]
2025/01/13 18:14:47 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 61.23
2025/01/13 18:14:47 INFO dspy.teleprompt.mipro_optimizer_v2: =============================


2025/01/13 18:14:47 INFO dspy.teleprompt.mipro_optimizer_v2: == Minibatch Trial 30 / 30 ==



Average Metric: 13.96 / 25 (55.8%): 100%|██████████| 25/25 [01:54<00:00,  4.60s/it]

2025/01/13 18:16:42 INFO dspy.evaluate.evaluate: Average Metric: 13.959038608628367 / 25 (55.8%)
2025/01/13 18:16:42 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 55.84 on minibatch of size 25 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 0'].
2025/01/13 18:16:42 INFO dspy.teleprompt.mipro_optimizer_v2: Minibatch scores so far: [53.12, 38.81, 56.4, 38.55, 0.0, 0.0, 40.33, 40.96, 52.79, 51.23, 57.84, 52.55, 47.41, 37.6, 0.0, 54.79, 46.45, 52.47, 41.81, 38.6, 50.38, 55.52, 49.66, 37.77, 50.51, 61.92, 47.61, 57.02, 36.04, 55.84]
2025/01/13 18:16:42 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [61.23, 53.33, 53.31]
2025/01/13 18:16:42 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 61.23
2025/01/13 18:16:42 INFO dspy.teleprompt.mipro_optimizer_v2: =============================


2025/01/13 18:16:42 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Full Eval 3 =====
2025/01/13 18:16:42 INFO dspy.teleprompt.mipro_optimizer_v2: 


Average Metric: 44.55 / 80 (55.7%): 100%|██████████| 80/80 [04:38<00:00,  3.48s/it]

2025/01/13 18:21:20 INFO dspy.evaluate.evaluate: Average Metric: 44.549296849117304 / 80 (55.7%)
2025/01/13 18:21:20 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [61.23, 53.33, 53.31, 55.69]
2025/01/13 18:21:20 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 61.23
2025/01/13 18:21:20 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/01/13 18:21:20 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/01/13 18:21:20 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 61.23!


In [128]:
compiled_program.save("perspective_model1.json")

In [129]:
loaded_program = Model()
loaded_program.load(path="perspective_model1.json")

In [85]:
"""model=Model()
metric_summary=SemanticF1_Summary(decompositional=False)
evaluator = Evaluate(devset=trainset[:100],max_errors=len(trainset[267:269]),
                    num_threads=1, display_progress=True,display_table=True
                    )
evaluator(model,metric=metric_summary)"""

"""model=Model()

model(**trainset[0].inputs())"""

'model=Model()\nmetric_summary=SemanticF1_Summary(decompositional=False)\nevaluator = Evaluate(devset=trainset[:100],max_errors=len(trainset[267:269]),\n                    num_threads=1, display_progress=True,display_table=True\n                    )\nevaluator(model,metric=metric_summary)'